In [ ]:
"""
Autor: Lucas Moreira
Projeto: ETL de dados Fenabrave (Novos e Seminovos/Usados)
Descrição: Notebook de análise dos dados pós-tratamento dos 50 Autos usados mais comercializados 
Data: 2025-06
"""

import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from gspread_dataframe import get_as_dataframe
import bar_chart_race as bcr
import warnings
warnings.filterwarnings("ignore")

/Users/lucasmoreira/.pyenv/versions/3.9.15/lib/python3.9/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.14) or chardet (5.2.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [2]:
# Caminho para seu arquivo JSON com a credencial
cred_path = '/Users/lucasmoreira/Documents/code/credentials.json'

# Definir escopos
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']

# Autenticação
creds = ServiceAccountCredentials.from_json_keyfile_name(cred_path, scope)
gc = gspread.authorize(creds)


# Abrir planilha e aba
spreadsheet_url = 'https://docs.google.com/spreadsheets/d/1NQ1O36AwIQk9rXCQOR7HTsnEmtegcwvVYTbpGlsco8Y/edit?gid=0#gid=0'
sh = gc.open_by_url(spreadsheet_url)
worksheet = sh.worksheet('Autos')
used_autos = get_as_dataframe(worksheet)

In [ ]:
# Ordena corretamente
used_autos = used_autos.sort_values(by=['model', 'ano_mes'])

# Formata ano_mes para o formato MM/YYYY
used_autos['periodo'] = used_autos['ano_mes'].apply(lambda x: f"{x[-2:]}/{x[:4]}")

# Cria coluna com o acumulado mês após mês
used_autos['total_acumulado'] = used_autos.groupby('model')['total'].cumsum()

# Transpõe a tabela de long para wide
pivot_used_autos = used_autos.pivot_table(
    index='ano_mes',
    columns='model',
    values='total_acumulado',
    aggfunc='sum'
).ffill()

period_map = {x: f"{x[-2:]}/{x[:4]}" for x in pivot_used_autos.index}
pivot_used_autos.index = pivot_used_autos.index.map(period_map)

# Seleciona os 20 modelos usados mais transacionados (a fins de visualização)
total_vendas_por_modelo = pivot_used_autos.iloc[-1]
top_modelos = total_vendas_por_modelo.sort_values(ascending=False).head(20).index
pivot_used_autos_bcr = pivot_used_autos[top_modelos]
pivot_used_autos_bcr = pivot_used_autos_bcr.fillna(0)

In [10]:
bcr.bar_chart_race(
    figsize=(5, 3),
    dpi=144,
    df=pivot_used_autos_bcr,
    filename='output/clip_used_fenabrave.mp4',
    cmap='tab20',
    title='Seminovos/Usados mais comercializados (#k)',
    title_size=12,
    bar_label_size=7,
    tick_label_size=7,
    shared_fontdict={'family' : 'Helvetica', 'color' : '.1'})